In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

### SERTI (EXTRAIDO EN EXCEL)

In [2]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\SERTI\*.csv',recursive=False)
serti_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (1).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (2).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (3).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (4).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (5).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (6).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (7).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (8).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (9).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (10).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (11).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (12).csv',


In [3]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])                     
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",nan,0,...,0,0,UNEFON,*******4162,"502,474.00",result.csv,nan,4162,2021-05-01,34205977_4162_7364_50.0
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",nan,0,...,0,0,TELCEL,*******0854,"458,139.00",result.csv,nan,0854,2021-05-01,34206011_0854_5366_100.0
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",nan,0,...,0,0,TELCEL,*******8827,"143,713.00",result.csv,nan,8827,2021-05-01,34206035_8827_7946_100.0
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",nan,0,...,0,0,TELCEL,*******7918,"19,708.00",result.csv,nan,7918,2021-05-01,34206062_7918_2697_50.0
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",nan,0,...,0,0,IUSACELL,*******1258,"503,435.00",result.csv,nan,1258,2021-05-01,34206091_1258_3500_50.0


In [4]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                   int64
carrier                      object
phone                        object
folio                        object
file_name                    object
claveVenta                  float64
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

In [5]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL
353733,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",nan,0,...,0,0,Unefon,9842888311,804866,result (19).csv,nan,8311,2021-06-20,37812948_8311_1149_100.0
353734,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",nan,0,...,0,0,Unefon,9842888311,804866,result (18).csv,nan,8311,2021-06-20,37812948_8311_1149_100.0
353735,6409,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",nan,0,...,0,0,Unefon,8993335289,804835,result (18).csv,nan,5289,2021-06-20,37812912_5289_7199_50.0
353736,6408,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",nan,0,...,0,0,Unefon,8993335289,804835,result (19).csv,nan,5289,2021-06-20,37812912_5289_7199_50.0
353737,6404,520416,0590,30.00,2021-06-20T00:00:31.474,37812952,Chip,"19,801.00",nan,0,...,0,0,Telcel,8445332124,731780,result (18).csv,nan,2124,2021-06-20,37812952_2124_0590_30.0


In [6]:
duplicates_serti.shape

(12818, 25)

In [7]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [8]:
serti.shape

(2100046, 25)

In [9]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                   int64
carrier                      object
phone                        object
folio                        object
file_name                    object
claveVenta                  float64
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

### DB PRODUCTO

In [10]:
name_file = ['merchant_id','topup_id','transaction_id','clip_external_tracking_id','carrier','phone_number','created_at',
             'updated_at','created_at_datetime','updated_at_datetime','updated_at_og','payment_type','status_code',
             'status_msg','amount','processor','issuer_auth_code','folio_id','receipt_no','last4','authNum','orderId',
             'serti_transaction_id','type','response_code']

In [11]:
producto_files_1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\TOPUPS\REVIEW\2022\01. ENERO\DATA\REVIEW TOPUPS 2022.01.27.txt',
                              sep='\t',names=name_file,dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
                                     'status_code':object,'serti_transaction_id':object})
# producto_files_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\11. NOVIEMBRE\DB\TOPUPS NOVIEMBRE 2021.csv',
#                               dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
#                                      'status_code':object})


In [12]:
# topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'phone_number':object,
#                                             'last4':object,'amount':float,'status_code':object})])
topups_db = pd.concat([producto_files_1])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['serti_transaction_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['serti_transaction_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,folio_id,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL
0,da4ed924-5091-4aeb-a876-fd8c64ab76f5,5a39bc48-fb09-44a4-adcb-eadf2f13bcbe,bf6e98fe-e457-4a4d-9cc1-6f1f7f44a1ed,sin_order_id_db,ATT,6461215663,2021-10-01,2021-10-01,2021-10-01 00:01:28,2021-10-01 00:01:28,...,nan,TU-Z2i4Zrp,1893,nan,41204798,sin_order_id_db,emv_sale,H009,5663,sin_order_id_db_5663_1893_200.0
1,3ee649c0-e6d9-4492-ad0c-482e15b6fc70,1210a460-eab7-4322-af15-762c6c7efb13,15f402d9-1eae-47ea-884b-239c816896e4,sin_order_id_db,TELCEL,7713236589,2021-10-01,2021-10-01,2021-10-01 00:01:51,2021-10-01 00:01:51,...,nan,TU-tSShuS5,7572,nan,41204806,sin_order_id_db,emv_sale,62,6589,sin_order_id_db_6589_7572_30.0
2,0952d88e-c307-4371-98ae-7de6b8398584,2b1e3abc-4131-4702-9aec-cfbcd0e1017b,17a7703e-feef-451f-9635-8130113389e5,sin_order_id_db,ATT,3323859948,2021-10-01,2021-10-01,2021-10-01 00:05:43,2021-10-01 00:05:43,...,nan,TU-Dpnea9X,4369,nan,41204855,sin_order_id_db,emv_sale,68,9948,sin_order_id_db_9948_4369_100.0
3,b5ab702b-7481-49ce-8772-5d2b5a000af7,302c6ca8-6d65-4ad7-9b73-8c09f60f1cc4,df3a1373-4ad8-47d2-93d1-bdd022bd8bd1,sin_order_id_db,MOVISTAR,9982634649,2021-10-01,2021-10-01,2021-10-01 00:07:28,2021-10-01 00:07:28,...,nan,TU-Z7PJgd9,5193,nan,41204871,sin_order_id_db,emv_sale,51,4649,sin_order_id_db_4649_5193_100.0
4,9505e71d-a955-49c3-8a12-1cca80dee256,5cdc8284-dbd5-47f1-b7a7-66e8f54cdbac,992ab446-d534-479d-a702-b268bfbf6255,sin_order_id_db,TELCEL,6311958885,2021-10-01,2021-10-01,2021-10-01 00:26:15,2021-10-01 00:26:15,...,nan,TU-aAdFvuS,1820,nan,41205040,sin_order_id_db,emv_sale,5,8885,sin_order_id_db_8885_1820_50.0


In [13]:
topups_db.shape

(91780, 27)

In [14]:
duplicates = topups_db[topups_db.duplicated(['topup_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,folio_id,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL
73223,49993af6-5a47-4676-aa9c-d4fbcdc695e3,615ef72a-8822-4df7-953a-cd3263cc8277,faed1195-f3f3-4d32-b9a2-dde1bc916357,sin_order_id_db,TELCEL,6682531417,2021-12-22,2021-12-22,2021-12-22 13:05:53,2021-12-22 13:06:48,...,nan,TU-93Wrtqo,0851,nan,45703228,sin_order_id_db,emv_sale,H006,1417,sin_order_id_db_1417_0851_100.0
73224,49993af6-5a47-4676-aa9c-d4fbcdc695e3,615ef72a-8822-4df7-953a-cd3263cc8277,faed1195-f3f3-4d32-b9a2-dde1bc916357,sin_order_id_db,TELCEL,6682531417,2021-12-22,2021-12-22,2021-12-22 13:05:53,2021-12-22 13:06:48,...,nan,TU-93Wrtqo,0851,"63,542.00",45703205,49350494,emv_sale,00,1417,49350494_1417_0851_100.0
73225,f317ab13-2a15-4f35-b1d9-09edf6466a4d,3fb80041-3936-44f2-8c70-a3e6df862d57,9d0e01a8-ac08-4ec8-a2d4-9b1729c5994b,sin_order_id_db,TELCEL,8110212786,2021-12-22,2021-12-22,2021-12-22 13:06:27,2021-12-22 13:06:48,...,nan,TU-lXWh3vG,1497,"6,867.00",45703204,49350496,emv_sale,00,2786,49350496_2786_1497_50.0
73226,f317ab13-2a15-4f35-b1d9-09edf6466a4d,3fb80041-3936-44f2-8c70-a3e6df862d57,9d0e01a8-ac08-4ec8-a2d4-9b1729c5994b,sin_order_id_db,TELCEL,8110212786,2021-12-22,2021-12-22,2021-12-22 13:06:27,2021-12-22 13:06:48,...,nan,TU-lXWh3vG,1497,nan,45703227,sin_order_id_db,emv_sale,H006,2786,sin_order_id_db_2786_1497_50.0


In [15]:
duplicates.shape

(4, 27)

In [16]:
duplicates['topup_id'].unique()

array(['615ef72a-8822-4df7-953a-cd3263cc8277',
       '3fb80041-3936-44f2-8c70-a3e6df862d57'], dtype=object)

In [17]:
topups_db.shape

(91780, 27)

In [18]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
created_at                   datetime64[ns]
updated_at                   datetime64[ns]
created_at_datetime                  object
updated_at_datetime                  object
updated_at_og                        object
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                    float64
folio_id                            float64
receipt_no                           object
last4                                object
authNum                             float64
orderId                               int64
serti_transaction_id            

### PIVOT DB

In [ ]:
pivot_db = pd.pivot_table(topups_db,index=['updated_at']
                            ,columns=['status_code']
                            ,values=['amount','serti_transaction_id']
                            ,aggfunc={'amount':np.sum,
                                     'serti_transaction_id':len},margins=True).reset_index()
pivot_db.fillna(0,inplace=True)
pivot_db.head()

In [ ]:
pivot_db.shape

### DB VS SERTI

In [ ]:
cruce_1 = topups_db.merge(serti[['INDEX_TOTAL','transactionId','panTerminacion','last4_phone_serti','serti_date','importe'
                                 ,'aprobada','carrier']]
                      ,on='INDEX_TOTAL',how='left')
cruce_1['aprobada'].fillna('Not_found',inplace=True)
cruce_1['transactionId'].fillna('Not_found',inplace=True)
cruce_1.head()

In [ ]:
cruce_1.shape

### ENCONTRADOS EN FILES DE SERTI

In [ ]:
cruce_1_found = cruce_1[(cruce_1['transactionId'] != 'Not_found')]
cruce_1_found['importe'].fillna(0,inplace=True)
cruce_1_found.head()

In [ ]:
cruce_1_found.shape

In [ ]:
cruce_1_found['aprobada'].unique()

In [ ]:
duplicates_cruce_1_found = cruce_1_found[cruce_1_found.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_found.head()

In [ ]:
duplicates_cruce_1_found.shape

### PIVOT ENCONTRADOS EN FILES DE SERTI

In [ ]:
pivot_cruce_1_found = pd.pivot_table(cruce_1_found,index=['updated_at']
                            ,columns=['aprobada']
                            ,values=['importe','amount','transactionId','serti_transaction_id']
                            ,aggfunc={'importe':np.sum,
                                     'amount':np.sum,
                                     'transactionId':len,
                                     'serti_transaction_id':len},margins=True).reset_index()
pivot_cruce_1_found.fillna(0,inplace=True)
pivot_cruce_1_found.head()

In [ ]:
pivot_cruce_1_found.shape

### NO ENCONTRADOS EN FILES DE SERTI

In [ ]:
cruce_1_not_found = cruce_1[(cruce_1['transactionId'] == 'Not_found')]
cruce_1_not_found['importe'].fillna(0,inplace=True)
cruce_1_not_found.head()

In [ ]:
cruce_1_not_found.shape

# EXCEL 

In [ ]:
writer = pd.ExcelWriter('CLIP-SERTI TOPUPS REVIEW 2022.01.27.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
cruce_1.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_found.to_excel(writer, sheet_name='FOUND',index=False)
pivot_cruce_1_found.to_excel(writer, sheet_name='PIVOT FOUND')
cruce_1_not_found.to_excel(writer, sheet_name='NOT FOUND',index=False)
writer.save()
writer.close()